In [1]:
import pandas as pd

In [2]:
data_path="data/datafinal.json"
submission_path="data/submission.csv"

# vars
col_text='Text'
col_cc='ContainsCode'
col_cl='CodeList'


In [13]:
full_data=pd.read_json(data_path)
train_data=full_data[full_data[col_cc]!=""]
test_data=full_data[full_data[col_cc]==""]
# pprint(train_data.iloc[1,1])
train_data
full_data.iloc[10]
# full_data.to_csv("/home/u131168/mh_shell/data/datafinal.csv")

ID                                                             11
Text            The field of software development is incredibl...
ContainsCode                                                 True
CodeList        #include <iostream> using namespace std; int m...
Name: 10, dtype: object

In [6]:
# !pip install tiktoken
# !pip install transformers
# model='Salesforce/codegen-350M-mono'
# model='Salesforce/codegen25-7b-instruct'
model="Phind/Phind-CodeLlama-34B-v1"
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model)


# from transformers import AutoTokenizer, LlamaForCausalLM
# model_path = "Phind/Phind-CodeLlama-34B-v1"
# model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_path)




ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [30]:
from pprint import pprint
# creating prompts
prompts=[]
for i in train_data[124:125].index:
    prompt='extract code from the following paragraph:\n\n\n'+train_data.iloc[i,1].replace('\n','')
    
    # predict
    # text = "def hello_world():"
    # text = prompt
    # input_ids = tokenizer(text, return_tensors="pt").input_ids
    # generated_ids = model.generate(input_ids, max_length=128)
    # res=tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)

    # Generate
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=256, do_sample=True, top_p=0.75, top_k=40, temperature=0.1)
    completion = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    completion = completion.replace(prompt, "").split("\n\n\n")[0]
    res=completion

    print('-----------------------text----------------')
    pprint(prompt)
    print('-------------------------res------------------')
    pprint(res)

    


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


-----------------------text----------------
('extract code from the following paragraph:\n'
 '\n'
 '\n'
 'DevOps is a set of practices that combines software development and IT '
 'operations. It aims to shorten the system development life cycle and provide '
 'continuous delivery with high software quality. DevOps is complementary with '
 'Agile software development; several DevOps aspects came from Agile '
 'methodology.')
-------------------------res------------------
('The problem is not a programming problem, but a text extraction problem. '
 'Here is a simple Python script that extracts the code from the given '
 'paragraph:\n'
 '\n'
 '```python\n'
 'paragraph = "DevOps is a set of practices that combines software development '
 'and IT operations. It aims to shorten the system development life cycle and '
 'provide continuous delivery with high software quality. DevOps is '
 'complementary with Agile software development; several DevOps aspects came '
 'from Agile methodology."\